<a href="https://colab.research.google.com/github/Davron030901/Machine_Learning/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data import



In [ ]:
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

adult = fetch_ucirepo(id=2)

X = adult.data.features
y = adult.data.targets

In [ ]:
# metadata
print(adult.metadata)

{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [ ]:
# variable information
print(adult.variables)

              name     role         type      demographic  \
0              age  Feature      Integer              Age   
1        workclass  Feature  Categorical           Income   
2           fnlwgt  Feature      Integer             None   
3        education  Feature  Categorical  Education Level   
4    education-num  Feature      Integer  Education Level   
5   marital-status  Feature  Categorical            Other   
6       occupation  Feature  Categorical            Other   
7     relationship  Feature  Categorical            Other   
8             race  Feature  Categorical             Race   
9              sex  Feature       Binary              Sex   
10    capital-gain  Feature      Integer             None   
11    capital-loss  Feature      Integer             None   
12  hours-per-week  Feature      Integer             None   
13  native-country  Feature  Categorical            Other   
14          income   Target       Binary           Income   

                       

In [ ]:
y['income'] = y['income'].replace({
    '>50K.': 1,
    '<=50K.': 0,
    '>50K': 1,
    '<=50K': 0
}).astype(np.int8)

<ipython-input-5-917d92965a37>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y['income'] = y['income'].replace({
<ipython-input-5-917d92965a37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['income'] = y['income'].replace({


#EDA

In [ ]:
edu = pd.concat([X['education'], X['education-num']], axis=1)
edu.groupby('education').mean()

,education-num
education,
10th,6.0
11th,7.0
12th,8.0
1st-4th,2.0
5th-6th,3.0
7th-8th,4.0
9th,5.0
Assoc-acdm,12.0
Assoc-voc,11.0


In [ ]:
Data=pd.concat([X, y], axis=1)

In [ ]:
Data.drop_duplicates(inplace=True)

In [ ]:
# Data.dropna(inplace=True)

In [ ]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48794 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48794 non-null  int64 
 1   workclass       47831 non-null  object
 2   fnlwgt          48794 non-null  int64 
 3   education       48794 non-null  object
 4   education-num   48794 non-null  int64 
 5   marital-status  48794 non-null  object
 6   occupation      47828 non-null  object
 7   relationship    48794 non-null  object
 8   race            48794 non-null  object
 9   sex             48794 non-null  object
 10  capital-gain    48794 non-null  int64 
 11  capital-loss    48794 non-null  int64 
 12  hours-per-week  48794 non-null  int64 
 13  native-country  48520 non-null  object
 14  income          48794 non-null  int8  
dtypes: int64(6), int8(1), object(8)
memory usage: 5.6+ MB


In [ ]:
Data.shape

(48794, 15)

In [ ]:
Data.isna().sum().sum()

2203

In [ ]:
Data.isna().sum()

,0
age,0
workclass,963
fnlwgt,0
education,0
education-num,0
marital-status,0
occupation,966
relationship,0
race,0
sex,0


In [ ]:
Data.fillna(method='bfill')

<ipython-input-14-9e4c8833b58a>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  Data.fillna(method='bfill')


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,0
48838,64,Private,321403,HS-grad,9,Widowed,Prof-specialty,Other-relative,Black,Male,0,0,40,United-States,0
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,0
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,0


In [ ]:
Data.ffill(inplace=True)

In [ ]:
Data.isna().sum()

,0
age,0
workclass,0
fnlwgt,0
education,0
education-num,0
marital-status,0
occupation,0
relationship,0
race,0
sex,0


In [ ]:
Data['native-country'].unique()

array(['United-States', 'Cuba', 'Jamaica', 'India', '?', 'Mexico',
       'South', 'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany',
       'Iran', 'Philippines', 'Italy', 'Poland', 'Columbia', 'Cambodia',
       'Thailand', 'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala',
       'China', 'Japan', 'Yugoslavia', 'Peru',
       'Outlying-US(Guam-USVI-etc)', 'Scotland', 'Trinadad&Tobago',
       'Greece', 'Nicaragua', 'Vietnam', 'Hong', 'Ireland', 'Hungary',
       'Holand-Netherlands'], dtype=object)

In [ ]:
# Find columns with '?' values
columns_with_question_marks = Data.columns[Data.isin(['?']).any()]

# Display rows containing '?' in those columns
rows_with_question_marks = Data[Data[columns_with_question_marks].isin(['?']).any(axis=1)]

rows_with_question_marks

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
14,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,?,1
27,54,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0,0,60,South,1
38,31,Private,84154,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,38,?,1
51,18,Private,226956,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,30,?,0
61,32,?,293936,7th-8th,4,Married-spouse-absent,?,Not-in-family,White,Male,0,0,40,?,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32530,35,?,320084,Bachelors,13,Married-civ-spouse,?,Wife,White,Female,0,0,55,United-States,1
32531,30,?,33811,Bachelors,13,Never-married,?,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,0
32539,71,?,287372,Doctorate,16,Married-civ-spouse,?,Husband,White,Male,0,0,10,United-States,1
32541,41,?,202822,HS-grad,9,Separated,?,Not-in-family,Black,Female,0,0,32,United-States,0


In [ ]:
columns_with_question_marks = Data.columns[(Data == '?').any()]

In [ ]:
columns_with_question_marks

Index(['workclass', 'occupation', 'native-country'], dtype='object')

In [ ]:
rows_without_question_marks = Data[~Data[columns_with_question_marks].isin(['?']).any(axis=1)]
rows_without_question_marks

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,0
48838,64,Private,321403,HS-grad,9,Widowed,Prof-specialty,Other-relative,Black,Male,0,0,40,United-States,0
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,0
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,0


In [ ]:
rows_without_question_marks['capital-loss'].value_counts()

,count
capital-loss,
0,44206
1902,296
1977,247
1887,229
2415,68
...,...
1539,1
4356,1
2080,1


In [ ]:
Data = rows_without_question_marks.drop(columns=['fnlwgt', 'education', 'capital-gain', 'capital-loss'])

#Data preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
ohe = OneHotEncoder(sparse_output=False, drop='first', dtype=np.int8)
scaler = StandardScaler()

ct = ColumnTransformer([
    ('o_enc', ohe, ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']),
    ('standard', scaler, ['age', 'hours-per-week', 'education-num'])
], remainder='passthrough').set_output(transform='pandas')

pre_1 = ct.fit_transform(Data)
pre_1

,o_enc__workclass_Local-gov,o_enc__workclass_Never-worked,o_enc__workclass_Private,o_enc__workclass_Self-emp-inc,o_enc__workclass_Self-emp-not-inc,o_enc__workclass_State-gov,o_enc__workclass_Without-pay,o_enc__marital-status_Married-AF-spouse,o_enc__marital-status_Married-civ-spouse,o_enc__marital-status_Married-spouse-absent,...,o_enc__native-country_Taiwan,o_enc__native-country_Thailand,o_enc__native-country_Trinadad&Tobago,o_enc__native-country_United-States,o_enc__native-country_Vietnam,o_enc__native-country_Yugoslavia,standard__age,standard__hours-per-week,standard__education-num,remainder__income
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0.032757,-0.061386,1.132962,0
1,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0.854397,-2.281733,1.132962,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,-0.041937,-0.061386,-0.432530,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1.078480,-0.061386,-1.215275,0
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,-0.788882,-0.061386,1.132962,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0.032757,-0.390326,1.132962,0
48838,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1.900120,-0.061386,-0.432530,0
48839,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,-0.041937,0.760965,1.132962,0
48840,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0.406230,-0.061386,1.132962,0


In [ ]:
pre_1.reset_index(inplace=True, drop=True)
pre_1

,o_enc__workclass_Local-gov,o_enc__workclass_Never-worked,o_enc__workclass_Private,o_enc__workclass_Self-emp-inc,o_enc__workclass_Self-emp-not-inc,o_enc__workclass_State-gov,o_enc__workclass_Without-pay,o_enc__marital-status_Married-AF-spouse,o_enc__marital-status_Married-civ-spouse,o_enc__marital-status_Married-spouse-absent,...,o_enc__native-country_Taiwan,o_enc__native-country_Thailand,o_enc__native-country_Trinadad&Tobago,o_enc__native-country_United-States,o_enc__native-country_Vietnam,o_enc__native-country_Yugoslavia,standard__age,standard__hours-per-week,standard__education-num,remainder__income
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0.032757,-0.061386,1.132962,0
1,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0.854397,-2.281733,1.132962,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,-0.041937,-0.061386,-0.432530,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1.078480,-0.061386,-1.215275,0
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,-0.788882,-0.061386,1.132962,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46391,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0.032757,-0.390326,1.132962,0
46392,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1.900120,-0.061386,-0.432530,0
46393,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,-0.041937,0.760965,1.132962,0
46394,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0.406230,-0.061386,1.132962,0


In [ ]:
X = pre_1.drop(columns=['remainder__income'])
y = pre_1['remainder__income']

In [ ]:
X.shape

(46396, 79)

In [ ]:
y.shape

(46396,)

In [ ]:
!pip install xgboost

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score
import matplotlib.pyplot as plt

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Voiting

In [ ]:
xgbc=XGBClassifier(n_estimators=100, learning_rate=0.1)
lr=LogisticRegression(max_iter=1000)
rfc=RandomForestClassifier(n_estimators=100)
svc=SVC(kernel='rbf', C=1.0,probability=True)
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, activation='relu', solver='adam')

mlp = MLPClassifier(
    hidden_layer_sizes=(50,),       
    max_iter=300,                   
    activation='relu',              
    solver='adam',                  
    alpha=0.0001,                   
    batch_size='auto',             
    learning_rate_init=0.001,       
    early_stopping=True,           
    validation_fraction=0.1,        
    n_iter_no_change=10,            
    verbose=True
    )

In [ ]:
models = {
    'LogisticRegression': lr,
    'RandomForestClassifier': rfc,
    'XGBClassifier': xgbc,
    'MLPClassifier': mlp,
    'SVC': svc
}

In [ ]:
print("Individual Model Performance:")
print("#" * 53)
individual_accuracies = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    individual_accuracies[name] = accuracy
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))
    print("#" * 53)

Individual Model Performance:
#####################################################
LogisticRegression Accuracy: 0.8316
              precision    recall  f1-score   support

           0       0.87      0.92      0.89      7069
           1       0.68      0.55      0.61      2211

    accuracy                           0.83      9280
   macro avg       0.77      0.73      0.75      9280
weighted avg       0.82      0.83      0.82      9280

#####################################################
RandomForestClassifier Accuracy: 0.8151
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      7069
           1       0.62      0.56      0.59      2211

    accuracy                           0.82      9280
   macro avg       0.75      0.73      0.74      9280
weighted avg       0.81      0.82      0.81      9280

#####################################################
XGBClassifier Accuracy: 0.8398
              precision    recall  f1-score   s

In [ ]:
modellar = [
    ('LogisticRegression', lr),
    ('RandomForestClassifier', rfc),
    ('XGBClassifier', xgbc),
    ('MLPClassifier', mlp),
    ('SVC', svc)
]

In [ ]:
hard_voting = VotingClassifier(
    estimators=modellar,
    voting='hard'
)
hard_voting.fit(X_train, y_train)

VotingClassifier(estimators=[('LogisticRegression',
                              LogisticRegression(max_iter=1000)),
                             ('RandomForestClassifier',
                              RandomForestClassifier()),
                             ('XGBClassifier',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_m...
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('MLPClassifier', MLPClassifier(max_iter=1000)),
                             ('SVC', SVC(probability=True))])

In [ ]:
hard_pred = hard_voting.predict(X_test)
hard_accuracy = accuracy_score(y_test, hard_pred)

print("Hard Voting Ensemble Performance:")
print("-" * 53)
print(f"Accuracy: {hard_accuracy:.4f}")
print(classification_report(y_test, hard_pred))
print("-" * 53)

Hard Voting Ensemble Performance:
------------------------------
Accuracy: 0.8383
              precision    recall  f1-score   support

           0       0.87      0.93      0.90      7069
           1       0.70      0.56      0.62      2211

    accuracy                           0.84      9280
   macro avg       0.79      0.74      0.76      9280
weighted avg       0.83      0.84      0.83      9280

------------------------------


In [ ]:
soft_voting = VotingClassifier(
    estimators=modellar,
    voting='soft',
    weights=[1, 0.8, 1, 0.7, 1]
)
soft_voting.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingClassifier(estimators=[('LogisticRegression',
                              LogisticRegression(max_iter=1000)),
                             ('RandomForestClassifier',
                              RandomForestClassifier()),
                             ('XGBClassifier',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_m...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('MLPClassifier',
                              MLPClassifier(hidden_layer_sizes=(50,),
                                            max_iter=300)),
                             ('SVC', SVC(probability=True))],
                 voting='soft', weights=[1, 0.8, 1, 0.7, 1])

In [ ]:
soft_pred = soft_voting.predict(X_test)
soft_accuracy = accuracy_score(y_test, soft_pred)

print("Soft Voting Ensemble Performance:")
print("-" * 53)
print(f"Accuracy: {soft_accuracy:.4f}")
print(classification_report(y_test, soft_pred))
print("-" * 53)

Soft Voting Ensemble Performance:
-----------------------------------------------------
Accuracy: 0.8375
              precision    recall  f1-score   support

           0       0.87      0.93      0.90      7069
           1       0.71      0.54      0.61      2211

    accuracy                           0.84      9280
   macro avg       0.79      0.74      0.76      9280
weighted avg       0.83      0.84      0.83      9280

-----------------------------------------------------


#Stacking

In [ ]:
model_1=LogisticRegression(max_iter=1000)
model_2=RandomForestClassifier(n_estimators=100)
model_3=XGBClassifier(n_estimators=100, learning_rate=0.1)
model_4=MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, activation='relu', solver='adam')
model_5=SVC(kernel='rbf', C=1.0,probability=True)

In [ ]:
all_models = [model_1, model_2, model_3,model_4, model_5]

In [ ]:
!pip install vecstack

  Preparing metadata (setup.py) ... done
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19861 sha256=59b51c312ac343ed3ebbdcab0fc223f198e2bc61b1e0f57dc4d1e2ae98c59565
  Stored in directory: /root/.cache/pip/wheels/88/3e/4b/a1a6ef703727d7f457db98b2a0b29fd3bd599c210d04639756
Successfully built vecstack


In [ ]:
from vecstack import stacking

In [ ]:
s_train, s_test = stacking(all_models, X_train,y_train, X_test,
                          regression=False,
                          shuffle=True,
                          random_state=42,
                          verbose=2)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [LogisticRegression]
    fold  0:  [0.83058519]
    fold  1:  [0.83004634]
    fold  2:  [0.83565039]
    fold  3:  [0.83090850]
    ----
    MEAN:     [0.83179761] + [0.00224563]
    FULL:     [0.83179761]

model  1:     [RandomForestClassifier]
    fold  0:  [0.81754499]
    fold  1:  [0.81549736]
    fold  2:  [0.81754499]
    fold  3:  [0.81366527]
    ----
    MEAN:     [0.81606315] + [0.00161723]
    FULL:     [0.81606315]

model  2:     [XGBClassifier]
    fold  0:  [0.84179330]
    fold  1:  [0.83759026]
    fold  2:  [0.83629701]
    fold  3:  [0.83963789]
    ----
    MEAN:     [0.83882962] + [0.00208487]
    FULL:     [0.83882962]

model  3:     [MLPClassifier]
    fold  0:  [0.82702877]
    fold  1:  [0.82638215]
    fold  2:  [0.82304128]
    fold  3:  [0.82444229]
    ----
    MEAN:     [0.82522362] + [0.00157907]
    FULL:     [0.8

In [ ]:
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(s_train, y_train)
meta_predictions = meta_model.predict(s_test)

In [ ]:
print('Meta-model accuracy: {:.4f}'.format(accuracy_score(y_test, meta_predictions)))
print(classification_report(y_test, meta_predictions))

Meta-model accuracy: 0.8391
              precision    recall  f1-score   support

           0       0.86      0.94      0.90      7069
           1       0.72      0.53      0.61      2211

    accuracy                           0.84      9280
   macro avg       0.79      0.73      0.75      9280
weighted avg       0.83      0.84      0.83      9280



#Blending

In [ ]:
train_ratio = 0.70
validation_ratio = 0.20
test_ratio = 0.10

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    X,y, test_size=1 - train_ratio)

In [ ]:
x_val, x_test, y_val, y_test = train_test_split(
    x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [ ]:
model_1.fit(x_train, y_train)
val_pred_1 = model_1.predict(x_val)
test_pred_1 = model_1.predict(x_test)

val_pred_1 = pd.DataFrame(val_pred_1)
test_pred_1 = pd.DataFrame(test_pred_1)

In [ ]:
model_2.fit(x_train, y_train)
val_pred_2 = model_2.predict(x_val)
test_pred_2 = model_2.predict(x_test)

# converting to dataframe
val_pred_2 = pd.DataFrame(val_pred_2)
test_pred_2 = pd.DataFrame(test_pred_2)

In [ ]:
model_3.fit(x_train, y_train)
val_pred_3 = model_3.predict(x_val)
test_pred_3 = model_3.predict(x_test)

# converting to dataframe
val_pred_3 = pd.DataFrame(val_pred_3)
test_pred_3 = pd.DataFrame(test_pred_3)

In [ ]:
model_4.fit(x_train, y_train)
val_pred_4 = model_4.predict(x_val)
test_pred_4 = model_4.predict(x_test)

# converting to dataframe
val_pred_4 = pd.DataFrame(val_pred_4)
test_pred_4 = pd.DataFrame(test_pred_4)

In [ ]:
model_5.fit(x_train, y_train)
val_pred_5 = model_5.predict(x_val)
test_pred_5 = model_5.predict(x_test)

# converting to dataframe
val_pred_5 = pd.DataFrame(val_pred_5)
test_pred_5 = pd.DataFrame(test_pred_5)

In [ ]:
df_val = pd.concat([x_val, val_pred_1, val_pred_2, val_pred_3, val_pred_4, val_pred_5], axis=1)
df_test = pd.concat([x_test, test_pred_1, test_pred_2, test_pred_3, test_pred_4, test_pred_5], axis=1)

In [ ]:
df_val.columns = df_val.columns.astype(str)
df_test.columns = df_test.columns.astype(str)

In [ ]:
final_model = LogisticRegression(max_iter=1000)
df_val_cleaned = df_val.dropna()
y_val_cleaned = y_val[df_val_cleaned.index]
final_model.fit(df_val_cleaned, y_val_cleaned)

LogisticRegression(max_iter=1000)

In [ ]:
df_test_cleaned = df_test.dropna()
y_test_cleaned = y_test[df_test_cleaned.index]
final_pred=final_model.predict(df_test_cleaned)

In [ ]:
print(classification_report(y_test_cleaned, final_pred))
print(accuracy_score(y_test_cleaned, final_pred))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       356
           1       0.65      0.59      0.62       104

    accuracy                           0.83       460
   macro avg       0.77      0.75      0.76       460
weighted avg       0.83      0.83      0.83       460

0.8347826086956521


#Bagging

In [ ]:
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = BaggingClassifier(
        estimator=DecisionTreeClassifier(),
        n_estimators=1,
        bootstrap=True,
        random_state=42
    )
model.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=1,
                  random_state=42)

In [ ]:
bagging_predictions = model.predict(X_test)

In [ ]:
print("\nBagging Results:")
print(f"Accuracy: {accuracy_score(y_test, bagging_predictions):.4f}")
print(classification_report(y_test, bagging_predictions))


Bagging Results:
Accuracy: 0.7784
              precision    recall  f1-score   support

           0       0.86      0.85      0.85      7069
           1       0.53      0.55      0.54      2211

    accuracy                           0.78      9280
   macro avg       0.70      0.70      0.70      9280
weighted avg       0.78      0.78      0.78      9280



#Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [ ]:
boosting_predictions = model.predict(X_test)

In [ ]:
print("\GradientBoostingClassifier Results:")
print(f"Accuracy: {accuracy_score(y_test, boosting_predictions):.4f}")
print(classification_report(y_test, boosting_predictions))

\GradientBoostingClassifier Results:
Accuracy: 0.8395
              precision    recall  f1-score   support

           0       0.87      0.93      0.90      7069
           1       0.70      0.56      0.63      2211

    accuracy                           0.84      9280
   macro avg       0.79      0.74      0.76      9280
weighted avg       0.83      0.84      0.83      9280

